## Summary
This notebook use an ensemble machine learning approach for classification. I train different models on different samples and get the average result of the predictions. Each model is a pipeline that consists of several steps. I use 5000 test samples with balanced classes to evaluate the performance of the models.

## Reading and exploration the Dataset

In [4]:
from IPython.display import display

import pandas as pd
import numpy as np
pd.set_option('display.max_row', 1000)
pd.set_option('display.max_colwidth', None)


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.model_selection import train_test_split

In [5]:
reviews = pd.read_csv('ar_reviews_100k.tsv', sep='\t')

In [6]:
reviews.sample(5)

,label,text
59756,Mixed,أمازلت تنامين على صوت ماجدة الرومي وتقولين لصوتها المنبعث من الراديو:. غن. غن يا ماجدة فلولا صوتك، لشعرت كم أنا وحيدة. وكم هن النساء بلا قلوب ... غن غن يا ماجدة، وليأت المطر والصحف اليومية والقهوة والحب والكلمات. ولأقول لكل النساء دمتن وافرات الظل وجميلات. ومن قلبي سلام لكل ولئك الذين لم يكبروا. ولن يكبروا إلى أكثر مما وصلوا إليه. الذين خطف الرصاص بصرهم مبكرا. إلى العصافير فوق وتحت كل ذرة تراب من الوطن. بإسم المطر ، والرقص .... بإسم البحر ، والذهول الأول .... اسميك نشيدي. وأنحني. كلما إنتهيت من قصيدة.. ما حلمك يا أبي:. أن أصلي الفجر في القدس، أطير لبيروت أسرق حجرا، أطير لدمشق أسرق وردة، أطير لبغداد أسرق كتابا، أطير لنيل القاهرة أسرق أغنية، أطير لفاس أسرق نقشا، أطير لتونس أسرق حمامة.. أحط في الجزائر، فتسرقني
80409,Negative,ضعيف جدا. لا كان السرير مزعج جدا. المكيف جدا حار و عند اخبار الطاقم لثلاث مرات لم يتجاوبو و لم يصلحوا الخلل، وكانت الغرفه جدا سيئة و رائحتها غير لائقه و بعدها خرجت من الفندق وحجز فندق اخر تقيمي العام من عشرهالفندق سئ جدا
56291,Mixed,مع إني أحب الرسائل وقراءتها جدا الا أنني لم أحب تحببه وتذلله لها في البداية. قصيرة ومبتورة لو اتبعت بردود غادة له لكان أفضل. الرسائل الأخيرة لإخته فائزة أصابتني في مقتل .
35288,Mixed,تم تغيير الغرفه الي الغرفه المطلوبه بعد جهد وقد تكررت هذه الحاله اكثر من مره . قربه من الحرم. لا يعترفون بنوع حجز الغرفه اي نوع السرير وكذاك المكان طلب لغير المدخنين
11534,Positive,مش هقدر اكتب ريفيو منصف للرواية لانى عندى افكار مختلطة حاليا بس اقدر اقول ان البداية خلت سقف توقعاتى عالى ويمكن عشان كده باقى ارواية وان كان جيد ولكن لم يكن المنشود بالنسبة لى. الحبكة الى حد بعيد جيدة ولكن عابها الاستطراد الممل فى غير موضع واسلوب د/عزالدين ليس الاسلوب الادبى المعتاد من كتاب كبار ك د/احمد خالد توفيق. مما يميز الرواية هو مضمونها واهمية ما تطرحه وبالتالى لا يوجد ادنى خسارة من قراءتها بلى على العكس ممكن ان تكون اسقراء للقادم وتحذير منه. ده رايى المبدئى عن الرواية :


In [5]:
reviews.describe(include='all')

,label,text
count,99999,99999
unique,3,99999
top,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشاطيء. المطعم
freq,33333,1


In [6]:
reviews['label'].value_counts()

Positive    33333
Mixed       33333
Negative    33333
Name: label, dtype: int64

Data is balanced and no null values

## Preprocessing

In [10]:
def label_encode(df):
#To convert leabels to int for some models
  return df['label'].replace({'Positive': 2, 'Negative': 1,'Mixed': 0}).astype('int')

Try different ways for preprocessing like stemming and removing stop words not help, they get down the accuracy so I remove maybe some critical terms like "ﻻ" to make the meaning different remove in the process 

## Baseline model (using pipeline)

In [11]:
#Note here I don't use the random seed to get more sense about data every train and train on many samples 
X_train1, X_test1, y_train1, y_test1 = train_test_split(reviews.text, reviews.label, test_size=0.05, 
                                                    stratify=reviews.label)


In [12]:
#Build pipe using CountVectorizer and LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

vec = CountVectorizer()
clf = LogisticRegression()
pipe = make_pipeline(vec, clf)
pipe.fit(X_train1,y_train1);

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
from sklearn import metrics

def print_report(pipe, x_test, y_test):
#To print classification report
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_report(pipe, X_test1, y_test1)

              precision    recall  f1-score   support

       Mixed       0.58      0.52      0.55      1667
    Negative       0.71      0.70      0.71      1666
    Positive       0.64      0.72      0.68      1667

    accuracy                           0.65      5000
   macro avg       0.65      0.65      0.64      5000
weighted avg       0.65      0.65      0.64      5000

accuracy: 0.647


In [14]:
# Another set of samples
X_train2, X_test2, y_train2, y_test2 = train_test_split(reviews.text, reviews.label, test_size=0.05, 
                                                    stratify=reviews.label)


In [15]:
#Build pipe_tfidf using TfidfVectorizer and LogisticRegression

vec = TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 5), min_df=.01, max_df=.3)
clf = LogisticRegression()
pipe_tfidf = make_pipeline(vec, clf)
pipe_tfidf.fit(X_train2,y_train2)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='char_wb', max_df=0.3, min_df=0.01,
                                 ngram_range=(3, 5))),
                ('logisticregression', LogisticRegression())])

In [16]:
print_report(pipe_tfidf, X_test2, y_test2)

              precision    recall  f1-score   support

       Mixed       0.62      0.56      0.59      1666
    Negative       0.70      0.74      0.72      1667
    Positive       0.71      0.74      0.72      1667

    accuracy                           0.68      5000
   macro avg       0.68      0.68      0.68      5000
weighted avg       0.68      0.68      0.68      5000

accuracy: 0.680


### Some predictions

In [17]:
pipe_tfidf.predict(['احلي مطعم في الدنيا'])

array(['Positive'], dtype=object)

In [18]:
pipe.predict(['اسوء مطعم في الدنيا'])

array(['Negative'], dtype=object)

In [19]:

for _, row in reviews.sample(5).iterrows():
    print(f"true label: {row['label']}")
    print(row['text'])
    display(pipe_tfidf.predict([row['text']]))
    print("--"*50)

true label: Negative
احد أقل كتب الدكتور احمد ..... مقدرتش افهم ايه الفكره او ايه المقصود من الكتاب كقصه يعتبر مقبول جدا جاد لكن لو في فكره مقصوده من الكتاب فهي فعلا ضلت الطريق الى عقلي ...او انني لم ارتق لأفهمها


array(['Negative'], dtype=object)

----------------------------------------------------------------------------------------------------
true label: Mixed
عبارة عن قصيدة تدور عن عبد لبنى الحسحاس اسمة سحيم تم قتله حرقا لتغزله في نساء القبيلة ويعيد القصيبي احياء الحكاية عن طريق قصيدته مع تطعيمها ببعض ابيات الشعر من تأليف سحيم نفسة


array(['Mixed'], dtype=object)

----------------------------------------------------------------------------------------------------
true label: Mixed
كل من لا يلتبس عليه الوضع السوري. يصبح مطعونا بمصداقيته. وكل من يبرر لظالم هو جاهل. وكل من يتعطش للفوضى هو مغامر. وكل من لا يصرخ بصوت أعلى من صوت السلفية. ليس بثائر حقيقي. لأن الحرية والثورة. لن تكونا على يد سلفية ولا عشائرية ولا رجعية. ستعيدنا مئات السنين إلى الوراء. ...من لم يحرر نفسه لا يستطيع أن يحرر وطنه. هكذا وصفت الكاتبة الثورة السورية حتى عام . أي ما قبل تفشي الداعشية. ثورة لها وجوه كثيرة. ورئيس له وجوه اخرى. فكتبت قانونها الاول:. أكره الجميع وأحب سوريا . حاولت ان لا تنتق لنفسها موقفا ولا موقعا ولا خطوطا حمراء. فوجدها البعض أنها ربما فقدت المصداقية. وبعد عامين من الطبعة الأولى. نردد معها ذات العبارة:. لا نريد حرية على شكل تقسيم. لكن....أليس هناك سواها من حرية؟


...

----------------------------------------------------------------------------------------------------
true label: Negative
لن اكرر نزولي في فندق بول مان زمزم مكه . . الغرف ليسة نظيفه و الاثاث قديم و الصاله مطله على الحمامات و المطبخ و ليست مطله على الحرم


array(['Negative'], dtype=object)

----------------------------------------------------------------------------------------------------
true label: Positive
محمود عزمي ... الشخصية المعقدة التي لا تجيد سوى جلد الذات. الابحار داخل محمود وصفحات حياته مرورا بالثورة العرابية ...الاحتلال الانجليزي ووصف الاجواء السياسية وقتها. نقلا لواحة سيوة وواقع مختلف وتفكير وعادات نجهل عنها الكثير. التحليق التاريخي المصاح للاحداث الاسكندر الاكبر واثار سيوة. في مهنيه وبراعة يجمع بهاء طاهر كل هذا في ملحمة ممتعة تستحق القراءة


array(['Positive'], dtype=object)

----------------------------------------------------------------------------------------------------


In [35]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(reviews.text, label_encode(reviews), test_size=0.05, 
                                                    stratify=label_encode(reviews))


## Improve the result

In [36]:
#Try using xgboost to improve with GPUs to run fast
import xgboost as xgb
from sklearn.pipeline import Pipeline

# Build the pipeline
pipe_tfidf2 = Pipeline([('tfidf', TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 5), min_df=.01, max_df=.3)), ('clf', xgb.XGBClassifier(enable_categorical=True,tree_method='gpu_hist'))])

pipe_tfidf2.fit(X_train3, y_train3)



Pipeline(steps=[('tfidf',
                 TfidfVectorizer(analyzer='char_wb', max_df=0.3, min_df=0.01,
                                 ngram_range=(3, 5))),
                ('clf',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=True, eval_metric=None,
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               objective='multi:softprob', predictor=None, ...))])

In [37]:
print_report(pipe_tfidf2, X_test3, y_test3)

              precision    recall  f1-score   support

           0       0.25      0.11      0.15      1667
           1       0.35      0.80      0.49      1666
           2       0.33      0.09      0.14      1667

    accuracy                           0.33      5000
   macro avg       0.31      0.33      0.26      5000
weighted avg       0.31      0.33      0.26      5000

accuracy: 0.335


Give a poor result :(

In [27]:
def proba(model,X_test):
#To get probabilities for each class
  return model.predict_proba(X_test)

In [40]:
# install catboost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 2.9 MB/s eta 0:00:00


In [38]:
#Note here need to label_encode function and also with xgboot
X_train4, X_test4, y_train4, y_test4 = train_test_split(reviews.text, label_encode(reviews), test_size=0.05, 
                                                    stratify=reviews.label)


In [45]:
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline

# Build pipline with GPUs
pipe_tfidf3 = Pipeline([('tfidf', TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 5), min_df=.01, max_df=.3)), ('clf', CatBoostClassifier(task_type='GPU', devices='0:1'))])

pipe_tfidf3.fit(X_train4, y_train4)



Learning rate set to 0.164338
0:	learn: 1.0530846	total: 419ms	remaining: 6m 58s
1:	learn: 1.0222771	total: 650ms	remaining: 5m 24s
2:	learn: 0.9983011	total: 874ms	remaining: 4m 50s
3:	learn: 0.9790112	total: 1.09s	remaining: 4m 31s
4:	learn: 0.9628982	total: 1.3s	remaining: 4m 18s
5:	learn: 0.9494355	total: 1.5s	remaining: 4m 8s
6:	learn: 0.9384312	total: 1.72s	remaining: 4m 3s
7:	learn: 0.9277454	total: 1.93s	remaining: 3m 59s
8:	learn: 0.9190731	total: 2.08s	remaining: 3m 49s
9:	learn: 0.9107079	total: 2.25s	remaining: 3m 42s
10:	learn: 0.9029011	total: 2.42s	remaining: 3m 37s
11:	learn: 0.8961816	total: 2.57s	remaining: 3m 31s
12:	learn: 0.8896944	total: 2.73s	remaining: 3m 27s
13:	learn: 0.8837161	total: 2.91s	remaining: 3m 25s
14:	learn: 0.8786111	total: 3.08s	remaining: 3m 21s
15:	learn: 0.8733914	total: 3.25s	remaining: 3m 19s
16:	learn: 0.8687744	total: 3.4s	remaining: 3m 16s
17:	learn: 0.8645149	total: 3.58s	remaining: 3m 15s
18:	learn: 0.8598168	total: 3.75s	remaining: 3m 1

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(analyzer='char_wb', max_df=0.3, min_df=0.01,
                                 ngram_range=(3, 5))),
                ('clf',
                 <catboost.core.CatBoostClassifier object at 0x7f2de1adde10>)])

In [46]:
print_report(pipe_tfidf3, X_test4, y_test4)

              precision    recall  f1-score   support

           0       0.62      0.56      0.59      1666
           1       0.72      0.72      0.72      1667
           2       0.70      0.76      0.73      1667

    accuracy                           0.68      5000
   macro avg       0.68      0.68      0.68      5000
weighted avg       0.68      0.68      0.68      5000

accuracy: 0.681


Catboost is the best single model result over all

In [66]:
X_train5, X_test5, y_train5, y_test5 = train_test_split(reviews.text, reviews.label, test_size=0.05, 
                                                    stratify=reviews.label)

In [69]:
#Build with LinearSVC
from sklearn.calibration import CalibratedClassifierCV
vec = TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 5), min_df=.01, max_df=.3)
clf = LinearSVC()
#Need that to get probability result for each class
calibrated = CalibratedClassifierCV(clf)
pipe_tfidf4 = make_pipeline(vec, calibrated)
pipe_tfidf4.fit(X_train5, y_train5)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='char_wb', max_df=0.3, min_df=0.01,
                                 ngram_range=(3, 5))),
                ('calibratedclassifiercv',
                 CalibratedClassifierCV(estimator=LinearSVC()))])

In [70]:
print_report(pipe_tfidf4, X_test5, y_test5)

              precision    recall  f1-score   support

       Mixed       0.60      0.53      0.57      1667
    Negative       0.70      0.74      0.72      1666
    Positive       0.68      0.73      0.70      1667

    accuracy                           0.67      5000
   macro avg       0.66      0.67      0.66      5000
weighted avg       0.66      0.67      0.66      5000

accuracy: 0.665


## Let's combine the all models 

Here we combine models and stack them and get mean for the prediction then get the final result

In [71]:
#Here you can add and remove piplines as you want I remove xgboost get a bad result and combine other piplines
#You can use any subset of data you want instead X_test1
avg_pr = np.stack([proba(pipe,X_test1)
,proba(pipe_tfidf,X_test1)
  , proba(pipe_tfidf3,X_test1)
  ,proba(pipe_tfidf4,X_test1)]).mean(0)
avg_pr.shape

(5000, 3)

In [72]:
#Get the index with the highest value as a prediction class
index = avg_pr.argmax(axis=1,)

In [73]:
def label_decode(index):
    return 'Positive' if index == 2 else 'Negative' if index == 1 else 'Mixed'

def print_final_report(index, y_test):
    #Get the final report for all stacked models
    y_pred = [label_decode(i) for i in index]
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_final_report(index, y_test1)

              precision    recall  f1-score   support

       Mixed       0.69      0.60      0.64      1667
    Negative       0.77      0.77      0.77      1666
    Positive       0.72      0.80      0.75      1667

    accuracy                           0.73      5000
   macro avg       0.72      0.73      0.72      5000
weighted avg       0.72      0.72      0.72      5000

accuracy: 0.725


There is a big improve in result using this technic it's like bagging but with different kind of models